# Capstone Project - The Battle of Neighborhoods

Note: A comprehensive report is seperately submitted. This notebook only contains the codes.

### Import all the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

####  Acquire the data from the following webpage. 
https://www.matthewproctor.com/australian_postcodes

In [2]:
df = pd.read_csv('australian_postcodes.csv')
df.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region
0,230,200,ANU,ACT,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R1
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,NaN,NaN,NaN,R1
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,Darwin City,701.0,Darwin,R1
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,Darwin City,701.0,Darwin,R1
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,Darwin Suburbs,701.0,Darwin,R1


Filter the data for the state of Victoria.

In [3]:
df = df[df['state'] == 'VIC']
df['council'] = df['sa3name']
df = df[['postcode', 'locality', 'long', 'lat', 'council']].reset_index(drop=True)
df.head()

,postcode,locality,long,lat,council
0,3000,MELBOURNE,144.956776,-37.817403,Melbourne City
1,3001,MELBOURNE,144.956776,-37.817403,Port Phillip
2,3002,EAST MELBOURNE,144.982207,-37.818517,Melbourne City
3,3003,WEST MELBOURNE,144.949592,-37.810871,Melbourne City
4,3004,MELBOURNE,144.970161,-37.844246,Port Phillip


Drop the duplicated suburbs

In [4]:
df = df.drop_duplicates(subset=['locality'])

In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.shape

(3339, 5)

Drop the rows having NaN values.

In [7]:
df.dropna(inplace=True)

Import the data of suburbs in Melbourne city.

In [8]:
sub_df = pd.read_csv('Suburbs_list.csv')
sub_df.head()

,Suburb
0,Abbotsford
1,Aberfeldie
2,Aintree
3,Airport West
4,Albanvale


Change the letters to upper case.

In [9]:
se = pd.Series(data=sub_df['Suburb'])

In [10]:
# create another column name locality
se = se.str.upper()
sub_df = pd.DataFrame(data=se)
sub_df['locality'] = sub_df['Suburb']
sub_df.head()

,Suburb,locality
0,ABBOTSFORD,ABBOTSFORD
1,ABERFELDIE,ABERFELDIE
2,AINTREE,AINTREE
3,AIRPORT WEST,AIRPORT WEST
4,ALBANVALE,ALBANVALE


Merge both the tables to extract the geographic information of only Melbourne's suburbs.

In [11]:
df1 = pd.merge(df, sub_df, how='inner', on='locality')
df1.head()

,postcode,locality,long,lat,council,Suburb
0,3000,MELBOURNE,144.956776,-37.817403,Melbourne City,MELBOURNE
1,3002,EAST MELBOURNE,144.982207,-37.818517,Melbourne City,EAST MELBOURNE
2,3003,WEST MELBOURNE,144.949592,-37.810871,Melbourne City,WEST MELBOURNE
3,3006,SOUTHBANK,144.965926,-37.823258,Port Phillip,SOUTHBANK
4,3008,DOCKLANDS,144.948039,-37.814719,Melbourne City,DOCKLANDS


In [12]:
df1.drop(['locality'], axis=1, inplace=True)


In [13]:
df1.shape

(516, 5)

Join the suburbs under single entry, having same postcode.

In [14]:
melb_df = df1.groupby('postcode')['Suburb'].apply(', '.join)

In [15]:
melb_df = pd.merge(df1, melb_df, how='right', on='postcode').reset_index(drop=True)
melb_df.head()

,postcode,long,lat,council,Suburb_x,Suburb_y
0,3000,144.956776,-37.817403,Melbourne City,MELBOURNE,MELBOURNE
1,3002,144.982207,-37.818517,Melbourne City,EAST MELBOURNE,EAST MELBOURNE
2,3003,144.949592,-37.810871,Melbourne City,WEST MELBOURNE,WEST MELBOURNE
3,3006,144.965926,-37.823258,Port Phillip,SOUTHBANK,SOUTHBANK
4,3008,144.948039,-37.814719,Melbourne City,DOCKLANDS,DOCKLANDS


In [16]:
melb_df.drop(['Suburb_x'], axis=1, inplace=True)

In [17]:
melb_df.drop_duplicates(subset='Suburb_y', inplace=True)

In [18]:
melb_df.rename(columns={'Suburb_y': 'suburb'}, inplace=True)

In [19]:
melb_df.head()

,postcode,long,lat,council,suburb
0,3000,144.956776,-37.817403,Melbourne City,MELBOURNE
1,3002,144.982207,-37.818517,Melbourne City,EAST MELBOURNE
2,3003,144.949592,-37.810871,Melbourne City,WEST MELBOURNE
3,3006,144.965926,-37.823258,Port Phillip,SOUTHBANK
4,3008,144.948039,-37.814719,Melbourne City,DOCKLANDS


In [20]:
melb_df = melb_df[['postcode', 'suburb', 'council', 'lat', 'long']]

In [21]:
melb_df.head()

,postcode,suburb,council,lat,long
0,3000,MELBOURNE,Melbourne City,-37.817403,144.956776
1,3002,EAST MELBOURNE,Melbourne City,-37.818517,144.982207
2,3003,WEST MELBOURNE,Melbourne City,-37.810871,144.949592
3,3006,SOUTHBANK,Port Phillip,-37.823258,144.965926
4,3008,DOCKLANDS,Melbourne City,-37.814719,144.948039


#### Disply the suburbs on a map

In [22]:
address = 'Melbourne, Victoria'

geolocator = Nominatim(user_agent="melbourne")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.8142176 144.9631608


In [23]:
map = folium.Map(location=[latitude, longitude], zoom_start=9) # generate map
# add a red circle marker to represent the Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Melbourne',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map)
# add the blue circle markers
for lat, lng, suburb, postcode, council in zip(melb_df.lat, melb_df.long, melb_df.suburb, melb_df.postcode, melb_df.council):
    label = '{}, {}, {}'.format(suburb, postcode, council)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map)

map

#### Import the Foursquare API to get the information about the venues in all the suburbs of Melbourne city.

In [24]:
CLIENT_ID = 'YBGLTNFMOZNETODM0FX1T0VVJTZGU5K1VRDDF3XZMGNJ3LFJ' # your Foursquare ID
CLIENT_SECRET = '4RKYKF1ZIQZN4YCFAKATFR2B1N11RNJTBQNCPVJV0CDYU3PU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YBGLTNFMOZNETODM0FX1T0VVJTZGU5K1VRDDF3XZMGNJ3LFJ
CLIENT_SECRET:4RKYKF1ZIQZN4YCFAKATFR2B1N11RNJTBQNCPVJV0CDYU3PU


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
melb_venues = getNearbyVenues(melb_df['suburb'], melb_df['lat'], melb_df['long'])

MELBOURNE
EAST MELBOURNE
WEST MELBOURNE
SOUTHBANK
DOCKLANDS
FOOTSCRAY, SEDDON
BROOKLYN, KINGSVILLE, MAIDSTONE, TOTTENHAM, WEST FOOTSCRAY
YARRAVILLE
NEWPORT, SOUTH KINGSVILLE, SPOTSWOOD
WILLIAMSTOWN, WILLIAMSTOWN NORTH
ALTONA, SEAHOLME
BRAYBROOK
ALBION, SUNSHINE, SUNSHINE NORTH, SUNSHINE WEST
ALBANVALE, KEALBA, KINGS PARK, ST ALBANS
ARDEER
BURNSIDE, BURNSIDE HEIGHTS, CAIRNLEA, CAROLINE SPRINGS, DEER PARK, RAVENHALL
FIELDSTONE, MAMBOURIN, MANOR LAKES, MOUNT COTTRELL, WYNDHAM VALE
ALTONA NORTH
LAVERTON NORTH
WILLIAMS LANDING
ALTONA MEADOWS, LAVERTON, SEABROOK
HOPPERS CROSSING, TARNEIT, TRUGANINA
COCOROC, DERRIMUT, POINT COOK, QUANDONG, WERRIBEE, WERRIBEE SOUTH
FLEMINGTON, KENSINGTON
ASCOT VALE, MARIBYRNONG, TRAVANCORE
KEILOR EAST
AVONDALE HEIGHTS
KEILOR, KEILOR NORTH
DELAHEY, HILLSIDE, PLUMPTON, SYDENHAM, TAYLORS HILL
KEILOR DOWNS, KEILOR LODGE, TAYLORS LAKES
MOONEE PONDS
ABERFELDIE, ESSENDON, ESSENDON WEST
ESSENDON NORTH, STRATHMORE, STRATHMORE HEIGHTS
AIRPORT WEST, KEILOR PARK, NIDDRIE


In [29]:
print(melb_venues.shape)
melb_venues.tail()

(2224, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2219,"RYE, ST ANDREWS BEACH, TOOTGAROOK",-38.396077,144.827203,The Dunes,-38.400314,144.825927,Golf Course
2220,SORRENTO,-38.345527,144.739101,Sorento Beach Motel,-38.345351,144.740123,Motel
2221,SORRENTO,-38.345527,144.739101,St Paul's Road General Store,-38.347290,144.743008,Café
2222,SORRENTO,-38.345527,144.739101,Fusion Broadband,-38.345672,144.744219,Business Service
2223,HAMPTON PARK,-38.043995,145.264296,Club Teutonia - German-Australian Club,-38.043995,145.261979,German Restaurant


Count the number of venues in each suburb.

In [30]:
melb_venues.groupby(by = 'Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ABBOTSFORD,21,21,21,21,21,21
"ABERFELDIE, ESSENDON, ESSENDON WEST",4,4,4,4,4,4
"AIRPORT WEST, KEILOR PARK, NIDDRIE",2,2,2,2,2,2
"ALBANVALE, KEALBA, KINGS PARK, ST ALBANS",7,7,7,7,7,7
"ALBERT PARK, MIDDLE PARK",24,24,24,24,24,24
"ALBION, SUNSHINE, SUNSHINE NORTH, SUNSHINE WEST",20,20,20,20,20,20
"ALPHINGTON, FAIRFIELD",4,4,4,4,4,4
"ALTONA MEADOWS, LAVERTON, SEABROOK",9,9,9,9,9,9
ALTONA NORTH,2,2,2,2,2,2


In [31]:
print('There are {} unique categories'.format(melb_venues['Venue Category'].nunique()))

There are 281 unique categories


#### Create a data frame having the frequency of each venues in a particular suburb

In [45]:
# one hot encoding
melb_onehot = pd.get_dummies(melb_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
melb_onehot['Neighborhood'] = melb_venues['Neighborhood']

melb_onehot.head()

,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo Exhibit,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MELBOURNE
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MELBOURNE
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MELBOURNE
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MELBOURNE
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,MELBOURNE


In [46]:
melb_grouped = melb_onehot.groupby(by = 'Neighborhood').mean().reset_index()
melb_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo Exhibit
0,ABBOTSFORD,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.047619,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
1,"ABERFELDIE, ESSENDON, ESSENDON WEST",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
2,"AIRPORT WEST, KEILOR PARK, NIDDRIE",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
3,"ALBANVALE, KEALBA, KINGS PARK, ST ALBANS",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.142857,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
4,"ALBERT PARK, MIDDLE PARK",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.041667,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
5,"ALBION, SUNSHINE, SUNSHINE NORTH, SUNSHINE WEST",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.050000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
6,"ALPHINGTON, FAIRFIELD",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
7,"ALTONA MEADOWS, LAVERTON, SEABROOK",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
8,ALTONA NORTH,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0
9,"ALTONA, SEAHOLME",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0


Frequecy of top five venues in each suburb

In [47]:
num_top_venues = 5

for hood in melb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = melb_grouped[melb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABBOTSFORD----
               venue  freq
0               Café  0.24
1                Pub  0.14
2   Greek Restaurant  0.05
3  Convenience Store  0.05
4        Coffee Shop  0.05


----ABERFELDIE, ESSENDON, ESSENDON WEST----
                  venue  freq
0  Gym / Fitness Center  0.25
1                  Café  0.25
2     Food & Drink Shop  0.25
3           Coffee Shop  0.25
4             Pawn Shop  0.00


----AIRPORT WEST, KEILOR PARK, NIDDRIE----
                  venue  freq
0  Gym / Fitness Center   0.5
1        Scenic Lookout   0.5
2        Adult Boutique   0.0
3                Office   0.0
4                  Park   0.0


----ALBANVALE, KEALBA, KINGS PARK, ST ALBANS----
                   venue  freq
0                 Bakery  0.14
1  Portuguese Restaurant  0.14
2          Grocery Store  0.14
3               Pharmacy  0.14
4  Vietnamese Restaurant  0.14


----ALBERT PARK, MIDDLE PARK----
                  venue  freq
0                 Beach  0.17
1                  Café  0.17
2     

4                Paintball Field   0.0


----CARLTON----
                                      venue  freq
0                                      Café  0.11
1                        Italian Restaurant  0.07
2                         Korean Restaurant  0.06
3                                       Pub  0.06
4  Residential Building (Apartment / Condo)  0.04


----CARLTON NORTH, PRINCES HILL----
            venue  freq
0            Café  0.29
1  Breakfast Spot  0.14
2    Tram Station  0.14
3   Grocery Store  0.07
4    Burger Joint  0.07


----CARNEGIE, GLEN HUNTLY, MURRUMBEENA----
          venue  freq
0    Playground  0.25
1   Coffee Shop  0.25
2  Tram Station  0.25
3          Park  0.25
4        Museum  0.00


----CARRUM DOWNS----
            venue  freq
0  Hardware Store  0.33
1     Flower Shop  0.33
2     Gas Station  0.33
3  Adult Boutique  0.00
4    Noodle House  0.00


----CARRUM, PATTERSON LAKES----
            venue  freq
0             Pub  0.33
1   Boat or Ferry  0.33
2  Clothing

                           venue  freq
0              German Restaurant   1.0
1                 Adult Boutique   0.0
2                   Noodle House   0.0
3  Paper / Office Supplies Store   0.0
4                Paintball Field   0.0


----HAMPTON, HAMPTON EAST----
               venue  freq
0               Café  0.31
1        Pizza Place  0.23
2             Bakery  0.08
3  French Restaurant  0.08
4        Supermarket  0.08


----HAWKSBURN, TOORAK----
            venue  freq
0             Spa  0.33
1     Supermarket  0.33
2        Pharmacy  0.33
3  Adult Boutique  0.00
4       Nightclub  0.00


----HAWTHORN----
                  venue  freq
0                  Café  0.15
1                   Bar  0.06
2  Fast Food Restaurant  0.06
3     Convenience Store  0.06
4      Ramen Restaurant  0.03


----HAWTHORN EAST----
               venue  freq
0               Café  0.25
1  Fish & Chips Shop  0.08
2    Thai Restaurant  0.08
3   Malay Restaurant  0.08
4               Park  0.08


----HEATHERTO

4                Pub  0.12


----NORTHCOTE----
                   venue  freq
0                   Café  0.20
1            Pizza Place  0.07
2  Vietnamese Restaurant  0.07
3                    Bar  0.07
4                    Pub  0.03


----OAKLEIGH SOUTH----
              venue  freq
0  Asian Restaurant  0.12
1       Supermarket  0.12
2        Baby Store  0.12
3          Pharmacy  0.12
4     Shopping Mall  0.12


----OFFICER, OFFICER SOUTH----
                        venue  freq
0  Construction & Landscaping   0.5
1               Train Station   0.5
2              Adult Boutique   0.0
3                Noodle House   0.0
4                        Park   0.0


----PAKENHAM, PAKENHAM SOUTH, PAKENHAM UPPER, RYTHDALE----
                           venue  freq
0            Arts & Crafts Store   1.0
1               Pedestrian Plaza   0.0
2                           Park   0.0
3  Paper / Office Supplies Store   0.0
4                Paintball Field   0.0


----PARK ORCHARDS----
                  

#### Create a data-frame having ten most common venues in each suburb

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = melb_grouped['Neighborhood']

for ind in np.arange(melb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(melb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABBOTSFORD,Café,Pub,Convenience Store,Japanese Restaurant,Park,Garden,Greek Restaurant,Grocery Store,Thrift / Vintage Store,Rock Climbing Spot
1,"ABERFELDIE, ESSENDON, ESSENDON WEST",Food & Drink Shop,Café,Gym / Fitness Center,Coffee Shop,Flea Market,Fishing Spot,Fish Market,Fish & Chips Shop,Filipino Restaurant,Egyptian Restaurant
2,"AIRPORT WEST, KEILOR PARK, NIDDRIE",Scenic Lookout,Gym / Fitness Center,Eastern European Restaurant,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
3,"ALBANVALE, KEALBA, KINGS PARK, ST ALBANS",Pharmacy,Bus Station,Grocery Store,Asian Restaurant,Vietnamese Restaurant,Bakery,Portuguese Restaurant,Fishing Spot,Flea Market,Fish Market
4,"ALBERT PARK, MIDDLE PARK",Café,Beach,Ice Cream Shop,Italian Restaurant,Pier,Fast Food Restaurant,Seafood Restaurant,Fishing Spot,Breakfast Spot,Supermarket


#### Create the clusters of suburbs

In [50]:
#set number of clusters
kclusters = 4

melb_grouped_clustering = melb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(melb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
melb_df.head()

,postcode,suburb,council,lat,long
0,3000,MELBOURNE,Melbourne City,-37.817403,144.956776
1,3002,EAST MELBOURNE,Melbourne City,-37.818517,144.982207
2,3003,WEST MELBOURNE,Melbourne City,-37.810871,144.949592
3,3006,SOUTHBANK,Port Phillip,-37.823258,144.965926
4,3008,DOCKLANDS,Melbourne City,-37.814719,144.948039


In [39]:
melb_df.rename(columns={'suburb': 'Neighborhood', 'council':'Council','lat':'Latitude', 'long':'Longitude'}, 
               inplace=True)
melb_df.drop('postcode', axis=1, inplace=True)

In [40]:
melb_df.head()

,Neighborhood,Council,Latitude,Longitude
0,MELBOURNE,Melbourne City,-37.817403,144.956776
1,EAST MELBOURNE,Melbourne City,-37.818517,144.982207
2,WEST MELBOURNE,Melbourne City,-37.810871,144.949592
3,SOUTHBANK,Port Phillip,-37.823258,144.965926
4,DOCKLANDS,Melbourne City,-37.814719,144.948039


In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

melb_merged = melb_df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
melb_merged = melb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

melb_merged # check the last columns!

,Neighborhood,Council,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MELBOURNE,Melbourne City,-37.817403,144.956776,0.0,Café,Coffee Shop,Korean Restaurant,Hotel,Japanese Restaurant,Burger Joint,Kebab Restaurant,Bagel Shop,Bar,Fast Food Restaurant
1,EAST MELBOURNE,Melbourne City,-37.818517,144.982207,0.0,Tennis Stadium,Cricket Ground,Café,Bar,Stadium,Restaurant,Football Stadium,Tennis Court,Tram Station,Tourist Information Center
2,WEST MELBOURNE,Melbourne City,-37.810871,144.949592,0.0,Pub,Café,Indian Restaurant,Hotel,General Entertainment,Gym / Fitness Center,Playground,Pizza Place,Sandwich Place,Asian Restaurant
3,SOUTHBANK,Port Phillip,-37.823258,144.965926,0.0,Bar,Café,Italian Restaurant,Theater,Grocery Store,Hotel,Australian Restaurant,Performing Arts Venue,Art Gallery,Coffee Shop
4,DOCKLANDS,Melbourne City,-37.814719,144.948039,0.0,Café,Coffee Shop,Hotel,Restaurant,Thai Restaurant,Sandwich Place,Bar,Pizza Place,Middle Eastern Restaurant,Australian Restaurant
5,"FOOTSCRAY, SEDDON",Maribyrnong,-37.807101,144.907953,0.0,Café,Eastern European Restaurant,Art Gallery,Gas Station,Sandwich Place,Gym,Brewery,Rental Car Location,Dessert Shop,Entertainment Service
7,"BROOKLYN, KINGSVILLE, MAIDSTONE, TOTTENHAM, WE...",Maribyrnong,-37.807135,144.861162,0.0,Food Truck,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Egyptian Restaurant
12,YARRAVILLE,Maribyrnong,-37.814183,144.888686,0.0,Café,Pizza Place,Grocery Store,Frozen Yogurt Shop,Burger Joint,Skate Park,Cambodian Restaurant,Movie Theater,Fast Food Restaurant,Thai Restaurant
13,"NEWPORT, SOUTH KINGSVILLE, SPOTSWOOD",Hobsons Bay,-37.838242,144.880556,0.0,Café,Pizza Place,Fast Food Restaurant,Liquor Store,Grocery Store,Bagel Shop,Convenience Store,Fish Market,Fish & Chips Shop,Filipino Restaurant
16,"WILLIAMSTOWN, WILLIAMSTOWN NORTH",Hobsons Bay,-37.863743,144.888461,0.0,Beach,Park,Italian Restaurant,Café,Athletics & Sports,Restaurant,Fish Market,Fish & Chips Shop,Fishing Spot,Filipino Restaurant


In [52]:
melb_merged.dropna(inplace=True)
melb_merged.reset_index(drop=True, inplace=True)
melb_merged['Cluster Labels']=melb_merged['Cluster Labels'].astype(int)

#### Display the clusters on the map

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melb_merged['Latitude'], melb_merged['Longitude'], melb_merged['Neighborhood'], melb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Create data-frame of each cluster.

In [54]:
cluster_1 = melb_merged.loc[melb_merged['Cluster Labels'] == 0, melb_merged.columns[[0] + list(range(5, melb_merged.shape[1]))]]
cluster_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MELBOURNE,Café,Coffee Shop,Korean Restaurant,Hotel,Japanese Restaurant,Burger Joint,Kebab Restaurant,Bagel Shop,Bar,Fast Food Restaurant
1,EAST MELBOURNE,Tennis Stadium,Cricket Ground,Café,Bar,Stadium,Restaurant,Football Stadium,Tennis Court,Tram Station,Tourist Information Center
2,WEST MELBOURNE,Pub,Café,Indian Restaurant,Hotel,General Entertainment,Gym / Fitness Center,Playground,Pizza Place,Sandwich Place,Asian Restaurant
3,SOUTHBANK,Bar,Café,Italian Restaurant,Theater,Grocery Store,Hotel,Australian Restaurant,Performing Arts Venue,Art Gallery,Coffee Shop
4,DOCKLANDS,Café,Coffee Shop,Hotel,Restaurant,Thai Restaurant,Sandwich Place,Bar,Pizza Place,Middle Eastern Restaurant,Australian Restaurant
5,"FOOTSCRAY, SEDDON",Café,Eastern European Restaurant,Art Gallery,Gas Station,Sandwich Place,Gym,Brewery,Rental Car Location,Dessert Shop,Entertainment Service
6,"BROOKLYN, KINGSVILLE, MAIDSTONE, TOTTENHAM, WE...",Food Truck,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Egyptian Restaurant
7,YARRAVILLE,Café,Pizza Place,Grocery Store,Frozen Yogurt Shop,Burger Joint,Skate Park,Cambodian Restaurant,Movie Theater,Fast Food Restaurant,Thai Restaurant
8,"NEWPORT, SOUTH KINGSVILLE, SPOTSWOOD",Café,Pizza Place,Fast Food Restaurant,Liquor Store,Grocery Store,Bagel Shop,Convenience Store,Fish Market,Fish & Chips Shop,Filipino Restaurant
9,"WILLIAMSTOWN, WILLIAMSTOWN NORTH",Beach,Park,Italian Restaurant,Café,Athletics & Sports,Restaurant,Fish Market,Fish & Chips Shop,Fishing Spot,Filipino Restaurant


In [55]:
cluster_2 = melb_merged.loc[melb_merged['Cluster Labels'] == 1, melb_merged.columns[[0] + list(range(5, melb_merged.shape[1]))]]
cluster_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,WILLIAMS LANDING,Playground,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
19,"COCOROC, DERRIMUT, POINT COOK, QUANDONG, WERRI...",Playground,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
24,"DELAHEY, HILLSIDE, PLUMPTON, SYDENHAM, TAYLORS...",Music Store,Playground,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
81,TEMPLESTOWE,Playground,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
172,KALORAMA,Playground,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


In [56]:
cluster_3= melb_merged.loc[melb_merged['Cluster Labels'] == 2, melb_merged.columns[[0] + list(range(5, melb_merged.shape[1]))]]
cluster_3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,MOONEE PONDS,Café,Japanese Restaurant,Bar,Supermarket,Greek Restaurant,Malay Restaurant,Stationery Store,Noodle House,Rest Area,Sports Bar
31,"PASCOE VALE, PASCOE VALE SOUTH",Café,Bakery,Supermarket,Australian Restaurant,Pizza Place,Field,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant
37,NORTH MELBOURNE,Café,Pub,Recreation Center,Pool,Football Stadium,Field,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market
40,"CARLTON NORTH, PRINCES HILL",Café,Tram Station,Breakfast Spot,Burger Joint,Grocery Store,Bakery,Cemetery,Wine Bar,Pub,Flea Market
54,THORNBURY,Café,Playground,Zoo Exhibit,Field,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
61,"IVANHOE, IVANHOE EAST",Café,Park,Health & Beauty Service,Home Service,Pet Store,Athletics & Sports,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant
71,YARRAMBAT,Café,Zoo Exhibit,Filipino Restaurant,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Electronics Store
88,CHIRNSIDE PARK,Café,Health & Beauty Service,Clothing Store,Zoo Exhibit,Fast Food Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Field
103,"HEATHMONT, RINGWOOD EAST",Café,Pizza Place,Park,Seafood Restaurant,Fishing Spot,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Egyptian Restaurant
110,ARMADALE,Café,Tram Station,Bridal Shop,Convenience Store,Japanese Restaurant,Train Station,Zoo Exhibit,Filipino Restaurant,Eye Doctor,Falafel Restaurant


In [57]:
cluster_4= melb_merged.loc[melb_merged['Cluster Labels'] == 3, melb_merged.columns[[0] + list(range(5, melb_merged.shape[1]))]]
cluster_4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,BRAYBROOK,Park,Recreation Center,Vietnamese Restaurant,Bakery,Flower Shop,Flea Market,Fishing Spot,Food,Fish Market,Egyptian Restaurant
25,"KEILOR DOWNS, KEILOR LODGE, TAYLORS LAKES",Park,Gym,Basketball Court,Café,Electronics Store,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
62,"BELLFIELD, HEIDELBERG HEIGHTS, HEIDELBERG WEST",Playground,Cosmetics Shop,Park,Field,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant
65,"EAGLEMONT, HEIDELBERG, ROSANNA, VIEWBANK",Park,Yoga Studio,Fish & Chips Shop,Zoo Exhibit,Fast Food Restaurant,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market
78,"BALWYN, DEEPDENE",Park,Movie Theater,Café,Italian Restaurant,Filipino Restaurant,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
80,BULLEEN,Trail,Athletics & Sports,Park,Zoo Exhibit,Fast Food Restaurant,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market
97,"BOX HILL NORTH, MONT ALBERT NORTH",Flower Shop,Park,Zoo Exhibit,Field,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
123,UPWEY,Park,Zoo Exhibit,Filipino Restaurant,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
126,"CARNEGIE, GLEN HUNTLY, MURRUMBEENA",Park,Tram Station,Playground,Coffee Shop,Egyptian Restaurant,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market
133,MULGRAVE,Park,Bakery,Café,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Zoo Exhibit


#### Observe each cluster for having Thai restaurant

In [58]:
col = cluster_1.columns
thai_1 = pd.concat((pd.DataFrame(cluster_1[cluster_1[i].isin(['Thai Restaurant'])], columns=cluster_1.columns)for i in col), ignore_index=True)
thai_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,WONGA PARK,Thai Restaurant,Tennis Court,Park,Grocery Store,Shopping Mall,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Entertainment Service,Eye Doctor
1,"ASHBURTON, ASHWOOD",Thai Restaurant,Pharmacy,Sandwich Place,Kebab Restaurant,Convenience Store,Liquor Store,Fish & Chips Shop,Fast Food Restaurant,Italian Restaurant,Park
2,"MACLEOD, YALLAMBIE",Café,Thai Restaurant,Train Station,Burger Joint,Fish & Chips Shop,Basketball Court,Park,Gym,Athletics & Sports,Flower Shop
3,"ALTONA, SEAHOLME",Convenience Store,Café,Train Station,Thai Restaurant,Fast Food Restaurant,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant,Farmers Market
4,"COBURG, COBURG NORTH",Café,Supermarket,Bakery,Thai Restaurant,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Shopping Mall,Food Truck,Fast Food Restaurant
5,"BUNDOORA, KINGSBURY",Supermarket,Pizza Place,Paper / Office Supplies Store,Thai Restaurant,Portuguese Restaurant,Flea Market,Fishing Spot,Fish Market,Fish & Chips Shop,Flower Shop
6,KNOXFIELD,Bakery,Grocery Store,Skate Park,Thai Restaurant,Zoo Exhibit,Farmers Market,Electronics Store,Entertainment Service,Eye Doctor,Falafel Restaurant
7,DOCKLANDS,Café,Coffee Shop,Hotel,Restaurant,Thai Restaurant,Sandwich Place,Bar,Pizza Place,Middle Eastern Restaurant,Australian Restaurant
8,MONTMORENCY,Pub,Grocery Store,Burger Joint,Café,Thai Restaurant,Park,Department Store,Electronics Store,Flea Market,Fishing Spot
9,MITCHAM,Pizza Place,Japanese Restaurant,Platform,Ice Cream Shop,Thai Restaurant,Gym / Fitness Center,Gym,Grocery Store,Fish & Chips Shop,Chinese Restaurant


In [59]:
col = cluster_2.columns
thai_2 = pd.concat((pd.DataFrame(cluster_2[cluster_2[i].isin(['Thai Restaurant'])], columns=col)for i in col), ignore_index=True)
thai_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [60]:
col = cluster_3.columns
thai_3 = pd.concat((pd.DataFrame(cluster_3[cluster_3[i].isin(['Thai Restaurant'])], columns=col)for i in col), ignore_index=True)
thai_3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [61]:
col = cluster_4.columns
thai_4 = pd.concat((pd.DataFrame(cluster_4[cluster_4[i].isin(['Thai Restaurant'])], columns=col)for i in col), ignore_index=True)
thai_4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### From cluster 1, extract the suburbs having Thai restaurant in their 5 most common venues.

In [62]:
top_5 = thai_1.iloc[:, 0:6]

In [63]:
col = top_5.columns
top_5_venues = pd.concat((pd.DataFrame(top_5[top_5[i].isin(['Thai Restaurant'])], columns=col)for i in col), ignore_index=True)
top_5_venues

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,WONGA PARK,Thai Restaurant,Tennis Court,Park,Grocery Store,Shopping Mall
1,"ASHBURTON, ASHWOOD",Thai Restaurant,Pharmacy,Sandwich Place,Kebab Restaurant,Convenience Store
2,"MACLEOD, YALLAMBIE",Café,Thai Restaurant,Train Station,Burger Joint,Fish & Chips Shop
3,"ALTONA, SEAHOLME",Convenience Store,Café,Train Station,Thai Restaurant,Fast Food Restaurant
4,"COBURG, COBURG NORTH",Café,Supermarket,Bakery,Thai Restaurant,Coffee Shop
5,"BUNDOORA, KINGSBURY",Supermarket,Pizza Place,Paper / Office Supplies Store,Thai Restaurant,Portuguese Restaurant
6,KNOXFIELD,Bakery,Grocery Store,Skate Park,Thai Restaurant,Zoo Exhibit
7,DOCKLANDS,Café,Coffee Shop,Hotel,Restaurant,Thai Restaurant
8,MONTMORENCY,Pub,Grocery Store,Burger Joint,Café,Thai Restaurant
9,MITCHAM,Pizza Place,Japanese Restaurant,Platform,Ice Cream Shop,Thai Restaurant


#### Insert the Latitue and Longitude into above dataframe.

In [64]:
selected_sub = pd.merge(top_5_venues, melb_df, how= 'inner', on='Neighborhood')

In [65]:
selected_sub = selected_sub.drop('Council', axis=1)
selected_sub

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Latitude,Longitude
0,WONGA PARK,Thai Restaurant,Tennis Court,Park,Grocery Store,Shopping Mall,-37.736734,145.267586
1,"ASHBURTON, ASHWOOD",Thai Restaurant,Pharmacy,Sandwich Place,Kebab Restaurant,Convenience Store,-37.867864,145.092946
2,"MACLEOD, YALLAMBIE",Café,Thai Restaurant,Train Station,Burger Joint,Fish & Chips Shop,-37.724398,145.069880
3,"ALTONA, SEAHOLME",Convenience Store,Café,Train Station,Thai Restaurant,Fast Food Restaurant,-37.861720,144.812736
4,"COBURG, COBURG NORTH",Café,Supermarket,Bakery,Thai Restaurant,Coffee Shop,-37.739901,144.964524
5,"BUNDOORA, KINGSBURY",Supermarket,Pizza Place,Paper / Office Supplies Store,Thai Restaurant,Portuguese Restaurant,-37.701346,145.056223
6,KNOXFIELD,Bakery,Grocery Store,Skate Park,Thai Restaurant,Zoo Exhibit,-37.893263,145.246731
7,DOCKLANDS,Café,Coffee Shop,Hotel,Restaurant,Thai Restaurant,-37.814719,144.948039
8,MONTMORENCY,Pub,Grocery Store,Burger Joint,Café,Thai Restaurant,-37.719759,145.124727
9,MITCHAM,Pizza Place,Japanese Restaurant,Platform,Ice Cream Shop,Thai Restaurant,-37.819222,145.196686


#### Display the suburbs on map, having Thai resturant in their 5 most common venues.

In [66]:
new_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map
# add a red circle marker to represent the Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Melbourne',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map)
# add the blue circle markers
for lat, lng, neigh in zip(selected_sub.Latitude, selected_sub.Longitude, selected_sub.Neighborhood):
    label = '{}'.format(neigh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup= label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(new_map)

new_map